### Monopoly

<p>In the game, <strong>Monopoly</strong>, the standard board is set up in the following way:</p>
<div class="center">
<img src="resources/images/0084_monopoly_board.png?1678992052" alt="0084_monopoly_board.png">
</div>
<p>A player starts on the GO square and adds the scores on two 6-sided dice to determine the number of squares they advance in a clockwise direction. Without any further rules we would expect to visit each square with equal probability: 2.5%. However, landing on G2J (Go To Jail), CC (community chest), and CH (chance) changes this distribution.</p>
<p>In addition to G2J, and one card from each of CC and CH, that orders the player to go directly to jail, if a player rolls three consecutive doubles, they do not advance the result of their 3rd roll. Instead they proceed directly to jail.</p>
<p>At the beginning of the game, the CC and CH cards are shuffled. When a player lands on CC or CH they take a card from the top of the respective pile and, after following the instructions, it is returned to the bottom of the pile. There are sixteen cards in each pile, but for the purpose of this problem we are only concerned with cards that order a movement; any instruction not concerned with movement will be ignored and the player will remain on the CC/CH square.</p>
<ul><li>Community Chest (2/16 cards):
<ol><li>Advance to GO</li>
<li>Go to JAIL</li>
</ol></li>
<li>Chance (10/16 cards):
<ol><li>Advance to GO</li>
<li>Go to JAIL</li>
<li>Go to C1</li>
<li>Go to E3</li>
<li>Go to H2</li>
<li>Go to R1</li>
<li>Go to next R (railway company)</li>
<li>Go to next R</li>
<li>Go to next U (utility company)</li>
<li>Go back 3 squares.</li>
</ol></li>
</ul><p>The heart of this problem concerns the likelihood of visiting a particular square. That is, the probability of finishing at that square after a roll. For this reason it should be clear that, with the exception of G2J for which the probability of finishing on it is zero, the CH squares will have the lowest probabilities, as 5/8 request a movement to another square, and it is the final square that the player finishes at on each roll that we are interested in. We shall make no distinction between "Just Visiting" and being sent to JAIL, and we shall also ignore the rule about requiring a double to "get out of jail", assuming that they pay to get out on their next turn.</p>
<p>By starting at GO and numbering the squares sequentially from 00 to 39 we can concatenate these two-digit numbers to produce strings that correspond with sets of squares.</p>
<p>Statistically it can be shown that the three most popular squares, in order, are JAIL (6.24%) = Square 10, E3 (3.18%) = Square 24, and GO (3.09%) = Square 00. So these three most popular squares can be listed with the six-digit modal string: 102400.</p>
<p>If, instead of using two 6-sided dice, two 4-sided dice are used, find the six-digit modal string.</p>


In [46]:
SQ_NAMES = [
    'GO', 'A1', 'CC1', 'A2', 'T1', 'R1', 'B1', 'CH1', 'B2', 'B3',
    'JAIL', 'C1', 'U1', 'C2', 'C3', 'R2', 'D1', 'CC2', 'D2', 'D3',
    'FP', 'E1', 'CH2', 'E2', 'E3', 'R3', 'F1', 'F2', 'U2', 'F3',
    'G2J', 'G1', 'G2', 'CC3', 'G3', 'R4', 'CH3', 'H1', 'T2', 'H2'
]

In [47]:
SQ_LOCATIONS = {n: l for l, n in enumerate(SQ_NAMES)}

In [48]:
import hashlib

def hash6(str_list):
    s = ','.join(str_list)
    return hashlib.md5(s.encode('utf-8')).hexdigest()[:6]

In [50]:
assert len(SQ_NAMES) == 40
assert hash6(SQ_NAMES) == 'c91376'

In [51]:
hash(','.join(SQ_NAMES[:5])) % 1000

540

In [55]:
import random

class Deck:
    def __init__(self, cards, seed=42):
        self.cards = cards
        random.Random(seed).shuffle(self.cards)
        self.idx = -1
        
    def draw_card(self):
        self.idx = (self.idx + 1) % len(self.cards)
        return self.cards[self.idx]
    
class CommunityChestDeck(Deck):
    def __init__(self, seed=42):
        super().__init__(['GO', 'JAIL'] + ['-'] * 14, seed)
    
    def draw_and_handle_card(self, loc):
        action = self.draw_card()
        if action in {'GO', 'JAIL'}:
            return SQ_LOCATIONS[action]
        else:
            return loc

In [56]:
cc = Deck(['GO', 'JAIL'] + ['-']*14)
assert len(cc.cards) == 16
hash6([cc.draw() for _ in range(100)])

AttributeError: 'Deck' object has no attribute 'draw'

In [59]:
# Community Chest deck
cc = Deck(['GO', 'JAIL'] + ['-'] * 14)
assert len(cc.cards) == 16
assert hash6([cc.draw_card() for _ in range(100)]) == 'db5854'

In [60]:
# Chance deck
ch = Deck(['GO', 'JAIL', 'C1', 'E3', 'H2', 'R1', 'next_R', 'next_R', 'next_U', 'back_3'] + ['-'] * 6)
assert len(ch.cards) == 16
assert hash6([ch.draw_card() for _ in range(100)]) == 'e8d6a2'

In [61]:
class Dice:
    def __init__(self, seed=42):
        self.rnd = random.Random(seed)

class Dice66(Dice):
    def roll(self):
        self.values = self.rnd.randint(1, 6), self.rnd.randint(1, 6)
        return sum(self.values)
    
    def is_double(self):
        return self.values[0] == self.values[1]

In [62]:
d = Dice66()
assert hash6([str(d.roll()) for _ in range(100)]) == 'bba13d'

In [65]:
class Simulation:
    def __init__(self, dice, seed=42):
        self.loc = 0
        self.cc = CommunityChestDeck(seed + 1)
        self.ch = Deck(
            ['GO', 'JAIL', 'C1', 'E3', 'H2', 'R1', 'next_R', 'next_R', 'next_U', 'back_3'] + ['-'] * 6,
            seed=seed + 2
        )
        self.double_counter = 0
        
        self.visits = {n: 0 for n in SQ_NAMES}
        self.dice = dice
        
    def step(self):
        # roll
        value = self.dice.roll()
        if self.dice.is_double(): self.double_counter += 1
        else: self.double_counter = 0
        
        if self.double_counter >= 3: # handle 3 consecutive doubles
            self.loc = SQ_LOCATIONS['JAIL']
        else:
            loc = (self.loc + value) % len(SQ_NAMES) # advance on board
            while loc != self.loc:
                self.loc = loc
                if SQ_NAMES[loc].startswith('CC'): # handle community chest deck
                    loc = self.cc.draw_and_handle_card(loc)
                elif SQ_NAMES[loc].startswith('CH'): # handle chance deck
                    sq = self.ch.draw_card()
                    if sq in {'GO', 'JAIL', 'C1', 'E3', 'H2', 'R1'}:
                        loc = SQ_LOCATIONS[sq]
                    elif sq == 'next_R':
                        while True:
                            loc = (loc + 1) % len(SQ_NAMES)
                            if SQ_NAMES[loc].startswith('R'): break
                    elif sq == 'next_U':
                        while True:
                            loc = (loc + 1) % len(SQ_NAMES)
                            if SQ_NAMES[loc].startswith('U'): break
                    elif sq == 'step_3':
                        loc = (loc - 3) % len(SQ_NAMES)
        self.visits[SQ_NAMES[self.loc]] += 1
            
    def run(self, nsteps=100_000):
        for i in range(nsteps):
            self.step()
            
sim = Simulation(Dice66())
sim.run()

In [66]:
import pandas as pd
se = pd.Series(sim.visits)
se = se / se.sum()
se.sort_values()[::-1][:3]

JAIL    0.03774
GO      0.03315
R1      0.03233
dtype: float64

In [45]:
# solution 2
from random import randint, shuffle
from collections import deque, Counter

squares = ["GO", "A1", "CC1", "A2", "T1", "R1", "B1", "CH1", "B2", "B3", "JAIL", "C1", "U1", "C2", "C3", "R2", "D1", "CC2", "D2", "D3", "FP", "E1", "CH2", "E2", "E3", "R3", "F1", "F2", "U2", "F3", "G2J", "G1", "G2", "CC3", "G3", "R4", "CH3", "H1", "T2", "H2"]
square_numbers = {square: i for i, square in enumerate(squares)}
num_squares = len(squares)

def build_deck(choices):
    numbers = [square_numbers[square] for square in choices]
    shuffle(numbers)
    return deque(numbers)

transitions = {
    "G2J": build_deck(["JAIL"]),
    "CH1": build_deck(["CH1", "CH1", "CH1", "CH1", "CH1", "CH1", "GO", "JAIL", "C1", "E3", "H2", "R1", "R2", "R2", "U1", "T1"]),
    "CH2": build_deck(["CH2", "CH2", "CH2", "CH2", "CH2", "CH2", "GO", "JAIL", "C1", "E3", "H2", "R1", "R2", "R3", "U2", "D3"]),
    "CH3": build_deck(["CH3", "CH3", "CH3", "CH3", "CH3", "CH3", "GO", "JAIL", "C1", "E3", "H2", "R1", "R1", "R1", "U1", "CC3"]),
    "CC1": build_deck(["CC1", "CC1", "CC1", "CC1", "CC1", "CC1", "CC1", "CC1", "CC1", "CC1", "CC1", "CC1", "CC1", "CC1", "GO", "JAIL"]),
    "CC2": build_deck(["CC2", "CC2", "CC2", "CC2", "CC2", "CC2", "CC2", "CC2", "CC2", "CC2", "CC2", "CC2", "CC2", "CC2", "GO", "JAIL"]),
    "CC3": build_deck(["CC3", "CC3", "CC3", "CC3", "CC3", "CC3", "CC3", "CC3", "CC3", "CC3", "CC3", "CC3", "CC3", "CC3", "GO", "JAIL"])
}
transitions = {square_numbers[square]: deck for square, deck in list(transitions.items())}

def transition(square_number):
    while square_number in transitions:
        deck = transitions[square_number]
        new_square_number = deck.pop()
        deck.appendleft(new_square_number)
        if square_number == new_square_number:
            break
        square_number = new_square_number
    return square_number

def move(square_number, die_size=6):
    roll = randint(1, die_size) + randint(1, die_size)
    square_number += roll
    square_number %= num_squares
    return transition(square_number)

def simulate(iterations=2000000, die_size=6):
    result = Counter()
    square_number = 0
    for i in range(iterations):
        square_number = move(square_number, die_size)
        result[square_number] += 1
    return result

def main():
    simulation = simulate(die_size=4)
    print(("".join("{:02}".format(square_number) for square_number, count in simulation.most_common(3))))

if __name__ == "__main__": main()

101524
